In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/goalie-stats/all-teams/45?qualified=all&conference=11&playertype=goalie&rookie=no&sort=wins&statstype=standard&page=1&league=1&division=-1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for i, row in enumerate(rows[1:], start=1):  
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for i, ele in enumerate(cols) if i != 0 and i != 2 and i != len(cols) - 1]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Jersey#', 'Name', 'Pos', 'Division', 'Team', 'GP', 'W', 'OTL', 'SOL', 'T', 'GA', 'GAA', 'SVS', 'SV%', 'SO', 'Min', 'SOW']

df = pd.DataFrame(acha_dynamic_stats, columns=column_names)

# Skip every other row, keeping only odd-indexed rows (1, 3, 5, etc.)
df = df.iloc[::2]
df = df[df['GP'].notna()]

numeric_columns = ['Jersey#', 'GP', 'W', 'OTL', 'SOL', 'T', 'GA', 'GAA', 'SVS', 'SV%', 'SO', 'Min', 'SOW']
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')  

string_columns = ['Name', 'Pos', 'Division', 'Team']
for column in string_columns:
    df[column] = df[column].astype(str)
    


print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_wd1_goalies.csv', index=False)

     Jersey#                   Name              Pos  \
0       79.0      Rachel Staresinic         WD1 WMCH   
2       30.0             Zoe Watson        WD1 WWCHL   
4       29.0  Rieley Jessie-Gerelli        WD1 CCWHA   
6       33.0         Meghan Stewart         WD1 WMCH   
8       35.0          Isabel Varner        WD1 CCWHA   
..       ...                    ...              ...   
190     31.0           Kaylee Esgar        WD1 WWCHL   
192     70.0          Sierra Briggs  WD1 Independent   
194      1.0           Eva Kingston        WD1 CCWHA   
196     29.0           Emily Carson        WD1 CCWHA   
198     48.0           Izzy Buckett        WD1 CCWHA   

                               Division Team  GP  W  OTL  SOL  T  GA    GAA  \
0              WD1 McKendree University   13  10  3    0    0  0  22   1.69   
2            WD1 University of Colorado   10   9  1    0    0  0  13   1.34   
4                    WD1 Adrian College   10   9  1    0    0  0  18   1.92   
6          